## 3 signal stacking

In [1]:
import numpy as np
import pandas as pd

from backtesting import Backtest, Strategy
from backtesting.lib import crossover

/usr/local/lib/python3.8/site-packages/backtesting/_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [2]:
#!pip install bokeh==2.4.2

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [4]:
df_price = (
    pd.read_csv('dowj_price_indicators.csv', index_col=0)
    .rename(columns={'Close*':'Close'})
    [['Date', 'Open', 'High', 'Low', 'Close']]
    .assign(Date = lambda df: pd.to_datetime(df.Date))
    .sort_values('Date')
    .set_index('Date', drop=True)
)

In [5]:
df_price

,Open,High,Low,Close
Date,,,,
2008-09-17,11056.58,11057.31,10595.90,10609.66
2008-09-18,10609.01,11076.44,10459.44,11019.69
2008-09-19,11027.51,11483.05,11026.70,11388.44
2008-09-22,11394.42,11394.58,10992.20,11015.69
2008-09-23,11015.69,11143.21,10833.94,10854.17
...,...,...,...,...
2016-06-27,17355.21,17355.21,17063.08,17140.24
2016-06-28,17190.51,17409.72,17190.51,17409.72
2016-06-29,17456.02,17704.51,17456.02,17694.68


In [6]:
df_predictions = (
    pd.read_csv('prediction_output.csv', index_col=0)
    .assign(Date = lambda df: pd.to_datetime(df.Date))
    .sort_values('Date')
    .set_index('Date', drop=True)
)

In [7]:
df_predictions

,iterate_prediction,grid_prediction,random_prediction
Date,,,
2014-03-05,-0.000488,0.000614,-0.002610
2014-03-06,0.004805,0.002387,-0.000320
2014-03-07,0.001993,0.002387,-0.000598
2014-03-10,0.003717,0.001717,-0.001000
2014-03-11,-0.001528,0.001017,-0.000465
...,...,...,...
2016-06-27,0.035532,0.005833,0.004592
2016-06-28,0.022988,0.012130,0.011819
2016-06-29,0.026804,0.005521,0.003258


In [8]:
# Predictions includes only testing data so the resulting df has only that as well
df_price_and_predictions = df_predictions.join(df_price)
df_price_and_predictions

,iterate_prediction,grid_prediction,random_prediction,Open,High,Low,Close
Date,,,,,,,
2014-03-05,-0.000488,0.000614,-0.002610,16395.88,16406.55,16343.96,16360.18
2014-03-06,0.004805,0.002387,-0.000320,16360.56,16450.17,16360.56,16421.89
2014-03-07,0.001993,0.002387,-0.000598,16424.53,16505.70,16398.86,16452.72
2014-03-10,0.003717,0.001717,-0.001000,16453.10,16453.10,16334.20,16418.68
2014-03-11,-0.001528,0.001017,-0.000465,16419.39,16460.33,16325.17,16351.25
...,...,...,...,...,...,...,...
2016-06-27,0.035532,0.005833,0.004592,17355.21,17355.21,17063.08,17140.24
2016-06-28,0.022988,0.012130,0.011819,17190.51,17409.72,17190.51,17409.72
2016-06-29,0.026804,0.005521,0.003258,17456.02,17704.51,17456.02,17694.68


In [9]:
def identity(x):
    '''Identity'''
    return x

class SignalStackingStrat1(Strategy):
    """
    Opens a long position whenever 2 or more prediction values
    are positive. Close it otherwise.
    """

    def init(self):
        self.p1 = self.I(identity, self.data.random_prediction)
        self.p2 = self.I(identity, self.data.iterate_prediction)
        self.p3 = self.I(identity, self.data.grid_prediction)

    def next(self):
        # Positive signals
        ps1 = int(self.p1[-1] >= 0)
        ps2 = int(self.p2[-1] >= 0)
        ps3 = int(self.p3[-1] >= 0)

        if (ps1 + ps2 + ps3) >= 2:
            self.buy()
        else:
            self.position.close()


In [10]:
# Creates backtest obj, runs and plots
bt = Backtest(df_price_and_predictions, SignalStackingStrat1, cash=100_000_000)#, exclusive_orders=True)
print(bt.run())
bt.plot()

Start                     2014-03-05 00:00:00
End                       2016-07-01 00:00:00
Duration                    849 days 00:00:00
Exposure Time [%]                   73.469388
Equity Final [$]                 112374609.33
Equity Peak [$]                  113294203.33
Return [%]                          12.374609
Buy & Hold Return [%]                9.713768
Return (Ann.) [%]                    5.127162
Volatility (Ann.) [%]               12.056266
Sharpe Ratio                         0.425269
Sortino Ratio                        0.639735
Calmar Ratio                         0.530425
Max. Drawdown [%]                   -9.666147
Avg. Drawdown [%]                   -2.331028
Max. Drawdown Duration      164 days 00:00:00
Avg. Drawdown Duration       39 days 00:00:00
# Trades                                  108
Win Rate [%]                        58.333333
Best Trade [%]                       3.587229
Worst Trade [%]                     -5.049272
Avg. Trade [%]                    

/usr/local/lib/python3.8/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/usr/local/lib/python3.8/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],


GridPlot(id='p1644', ...)

In [11]:
def identity(x):
    '''Identity'''
    return x

class SignalStackingStrat2(Strategy):
    """
    Open a long position whenever the 3 prediction values are positive.
    Open a short position whenever the 3 prediction values are negative.
    """

    def init(self):
        self.p1 = self.I(identity, self.data.random_prediction)
        self.p2 = self.I(identity, self.data.iterate_prediction)
        self.p3 = self.I(identity, self.data.grid_prediction)

    def next(self):
        # Positive signals
        ps1 = int(self.p1[-1] >= 0)
        ps2 = int(self.p2[-1] >= 0)
        ps3 = int(self.p3[-1] >= 0)

        # Negative signals
        ns1 = int(self.p1[-1] < 0)
        ns2 = int(self.p2[-1] < 0)
        ns3 = int(self.p3[-1] < 0)

        if (ps1 + ps2 + ps3) == 3:
            self.buy()
        elif (ns1 + ns2 + ns3) == 3:
            self.sell()


In [12]:
# Creates backtest obj, runs and plots
bt = Backtest(df_price_and_predictions, SignalStackingStrat2, cash=100_000_000, exclusive_orders=True)
print(bt.run())
bt.plot()

Start                     2014-03-05 00:00:00
End                       2016-07-01 00:00:00
Duration                    849 days 00:00:00
Exposure Time [%]                   97.959184
Equity Final [$]                 125937480.41
Equity Peak [$]                  128048092.25
Return [%]                           25.93748
Buy & Hold Return [%]                9.713768
Return (Ann.) [%]                   10.388434
Volatility (Ann.) [%]                15.22537
Sharpe Ratio                         0.682311
Sortino Ratio                        1.111478
Calmar Ratio                         0.884584
Max. Drawdown [%]                  -11.743865
Avg. Drawdown [%]                   -2.521663
Max. Drawdown Duration      251 days 00:00:00
Avg. Drawdown Duration       34 days 00:00:00
# Trades                                  247
Win Rate [%]                        54.251012
Best Trade [%]                       3.043256
Worst Trade [%]                     -4.640792
Avg. Trade [%]                    

/usr/local/lib/python3.8/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/usr/local/lib/python3.8/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],


GridPlot(id='p3028', ...)

## how to start your own project

In [15]:
#pip install yfinance

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
     |████████████████████████████████| 65 kB 3.5 MB/s             
  Using cached frozendict-2.3.8-cp38-cp38-macosx_10_9_x86_64.whl (35 kB)
  Using cached pytz-2023.3-py2.py3-none-any.whl (502 kB)
  Using cached appdirs-1.4.4-py2.py3-none-any.whl (9.6 kB)
  Using cached html5lib-1.1-py2.py3-none-any.whl (112 kB)
  Using cached multitasking-0.0.11-py3-none-any.whl (8.5 kB)
  Using cached beautifulsoup4-4.12.2-py3-none-any.whl (142 kB)
  Using cached requests-2.31.0-py3-none-any.whl (62 kB)
  Attempting uninstall: pytz
    Found existing installation: pytz 2022.1
    Uninstalling pytz-2022.1:
      Successfully uninstalled pytz-2022.1
  DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will 

In [16]:
import yfinance as yf


In [17]:
msft = yf.Ticker("MSFT")


In [18]:
msft.info

{'address1': 'One Microsoft Way',
 'city': 'Redmond',
 'state': 'WA',
 'zip': '98052-6399',
 'country': 'United States',
 'phone': '425 882 8080',
 'website': 'https://www.microsoft.com',
 'industry': 'Software—Infrastructure',
 'industryDisp': 'Software—Infrastructure',
 'sector': 'Technology',
 'sectorDisp': 'Technology',
 'longBusinessSummary': 'Microsoft Corporation develops and supports software, services, devices and solutions worldwide. The Productivity and Business Processes segment offers office, exchange, SharePoint, Microsoft Teams, office 365 Security and Compliance, Microsoft viva, and Microsoft 365 copilot; and office consumer services, such as Microsoft 365 consumer subscriptions, Office licensed on-premises, and other office services. This segment also provides LinkedIn; and dynamics business solutions, including Dynamics 365, a set of intelligent, cloud-based applications across ERP, CRM, power apps, and power automate; and on-premises ERP and CRM applications. The Int

In [19]:
hist = msft.history(period="120mo")
display(hist)

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2013-08-19 00:00:00-04:00,26.596290,26.772146,26.278071,26.286446,27902500,0.00,0.0
2013-08-20 00:00:00-04:00,26.328317,26.713527,26.269698,26.479052,22979600,0.00,0.0
2013-08-21 00:00:00-04:00,26.470675,26.805638,26.412056,26.470675,37409100,0.00,0.0
2013-08-22 00:00:00-04:00,26.956374,27.207601,26.881007,27.123857,31169900,0.00,0.0
2013-08-23 00:00:00-04:00,29.451869,29.476994,28.472096,29.100157,225493800,0.00,0.0
...,...,...,...,...,...,...,...
2023-08-14 00:00:00-04:00,320.711012,323.375354,319.403752,323.355408,18836100,0.00,0.0
2023-08-15 00:00:00-04:00,322.317568,324.403149,320.221999,321.179962,16966300,0.00,0.0
2023-08-16 00:00:00-04:00,320.799988,324.420013,319.799988,320.399994,20698900,0.68,0.0


In [20]:
# show meta information about the history (requires history() to be called first)
msft.history_metadata


{'currency': 'USD',
 'symbol': 'MSFT',
 'exchangeName': 'NMS',
 'instrumentType': 'EQUITY',
 'firstTradeDate': 511108200,
 'regularMarketTime': 1692388801,
 'gmtoffset': -14400,
 'timezone': 'EDT',
 'exchangeTimezoneName': 'America/New_York',
 'regularMarketPrice': 316.48,
 'chartPreviousClose': 31.8,
 'priceHint': 2,
 'currentTradingPeriod': {'pre': {'timezone': 'EDT',
   'end': 1692624600,
   'start': 1692604800,
   'gmtoffset': -14400},
  'regular': {'timezone': 'EDT',
   'end': 1692648000,
   'start': 1692624600,
   'gmtoffset': -14400},
  'post': {'timezone': 'EDT',
   'end': 1692662400,
   'start': 1692648000,
   'gmtoffset': -14400}},
 'dataGranularity': '1d',
 'range': '120mo',
 'validRanges': ['1d',
  '5d',
  '1mo',
  '3mo',
  '6mo',
  '1y',
  '2y',
  '5y',
  '10y',
  'ytd',
  'max']}

In [21]:
display(msft.actions)


,Dividends,Stock Splits
Date,,
2013-11-19 00:00:00-05:00,0.28,0.0
2014-02-18 00:00:00-05:00,0.28,0.0
2014-05-13 00:00:00-04:00,0.28,0.0
2014-08-19 00:00:00-04:00,0.28,0.0
2014-11-18 00:00:00-05:00,0.31,0.0
2015-02-17 00:00:00-05:00,0.31,0.0
2015-05-19 00:00:00-04:00,0.31,0.0
2015-08-18 00:00:00-04:00,0.31,0.0
2015-11-17 00:00:00-05:00,0.36,0.0


In [22]:
msft.dividends



Date
2013-11-19 00:00:00-05:00    0.28
2014-02-18 00:00:00-05:00    0.28
2014-05-13 00:00:00-04:00    0.28
2014-08-19 00:00:00-04:00    0.28
2014-11-18 00:00:00-05:00    0.31
2015-02-17 00:00:00-05:00    0.31
2015-05-19 00:00:00-04:00    0.31
2015-08-18 00:00:00-04:00    0.31
2015-11-17 00:00:00-05:00    0.36
2016-02-16 00:00:00-05:00    0.36
2016-05-17 00:00:00-04:00    0.36
2016-08-16 00:00:00-04:00    0.36
2016-11-15 00:00:00-05:00    0.39
2017-02-14 00:00:00-05:00    0.39
2017-05-16 00:00:00-04:00    0.39
2017-08-15 00:00:00-04:00    0.39
2017-11-15 00:00:00-05:00    0.42
2018-02-14 00:00:00-05:00    0.42
2018-05-16 00:00:00-04:00    0.42
2018-08-15 00:00:00-04:00    0.42
2018-11-14 00:00:00-05:00    0.46
2019-02-20 00:00:00-05:00    0.46
2019-05-15 00:00:00-04:00    0.46
2019-08-14 00:00:00-04:00    0.46
2019-11-20 00:00:00-05:00    0.51
2020-02-19 00:00:00-05:00    0.51
2020-05-20 00:00:00-04:00    0.51
2020-08-19 00:00:00-04:00    0.51
2020-11-18 00:00:00-05:00    0.56
2021-02-1

In [23]:
msft.splits

Series([], Name: Stock Splits, dtype: float64)

In [24]:
# show share count
msft.get_shares_full(start="2022-01-01", end=None)


2022-01-27 00:00:00-05:00    7496869888
2022-02-04 00:00:00-05:00    7800719872
2022-02-05 00:00:00-05:00    7496869888
2022-02-11 00:00:00-05:00    7496869888
2022-03-04 00:00:00-05:00    7605040128
                                ...    
2023-07-07 00:00:00-04:00    7435489792
2023-07-12 00:00:00-04:00    7449829888
2023-07-13 00:00:00-04:00    7435489792
2023-07-19 00:00:00-04:00    7731420160
2023-07-21 00:00:00-04:00    7435489792
Length: 61, dtype: int64

In [25]:
# - income statement
msft.income_stmt



,2023-06-30,2022-06-30,2021-06-30,2020-06-30
Tax Effect Of Unusual Items,0.0,0.0,0.0,0.0
Tax Rate For Calcs,0.189786,0.131134,0.138266,0.16
Normalized EBITDA,102384000000.0,97843000000.0,81602000000.0,65755000000.0
Total Unusual Items,NaN,334000000.0,1303000000.0,28000000.0
Total Unusual Items Excluding Goodwill,NaN,334000000.0,1303000000.0,28000000.0
Net Income From Continuing Operation Net Minority Interest,72361000000.0,72738000000.0,61271000000.0,44281000000.0
Reconciled Depreciation,13861000000.0,14460000000.0,11686000000.0,12796000000.0
Reconciled Cost Of Revenue,65863000000.0,62650000000.0,52232000000.0,46078000000.0
EBIT,88523000000.0,83383000000.0,69916000000.0,52959000000.0
Net Interest Income,NaN,31000000.0,-215000000.0,89000000.0


In [26]:
msft.quarterly_income_stmt

,2023-06-30,2023-03-31,2022-12-31,2022-09-30,2022-06-30
Tax Effect Of Unusual Items,0.0,0.0,0.0,-10640000.0,0.0
Tax Rate For Calcs,0.19,0.192917,0.192438,0.19,0.182896
Normalized EBITDA,28128000000.0,25901000000.0,24047000000.0,24918000000.0,24513000000.0
Total Unusual Items,NaN,162000000.0,-232000000.0,-56000000.0,-80000000.0
Total Unusual Items Excluding Goodwill,NaN,162000000.0,-232000000.0,-56000000.0,-80000000.0
Net Income From Continuing Operation Net Minority Interest,20081000000.0,18299000000.0,16425000000.0,17556000000.0,16740000000.0
Reconciled Depreciation,3874000000.0,3549000000.0,3648000000.0,2790000000.0,3979000000.0
Reconciled Cost Of Revenue,16795000000.0,16128000000.0,17488000000.0,15452000000.0,16429000000.0
EBIT,24254000000.0,22352000000.0,20399000000.0,22072000000.0,20534000000.0
Net Interest Income,NaN,252000000.0,210000000.0,141000000.0,56000000.0


In [27]:
# show holders
display(msft.major_holders)
display(msft.institutional_holders)
display(msft.mutualfund_holders)


,0,1
0,0.05%,% of Shares Held by All Insider
1,73.21%,% of Shares Held by Institutions
2,73.25%,% of Float Held by Institutions
3,6255,Number of Institutions Holding Shares


,Holder,Shares,Date Reported,% Out,Value
0,Vanguard Group Inc,653247546,2023-06-29,0.0879,206739790534
1,Blackrock Inc.,536245027,2023-06-29,0.0722,169710832036
2,State Street Corporation,293614078,2023-06-29,0.0395,92922986631
3,"FMR, LLC",210111250,2023-06-29,0.0283,66496010708
4,Price (T.Rowe) Associates Inc,159904219,2023-06-29,0.0215,50606488985
5,"Geode Capital Management, LLC",148129470,2023-06-29,0.0199,46880016292
6,Morgan Stanley,123498322,2023-06-29,0.0166,39084750303
7,JP Morgan Chase & Company,113024837,2023-06-29,0.0152,35770101655
8,Norges Bank Investment Management,86316926,2022-12-30,0.0116,27317581688
9,Capital World Investors,85507261,2023-06-29,0.0115,27061338900


,Holder,Shares,Date Reported,% Out,Value
0,Vanguard Total Stock Market Index Fund,226594602,2022-12-30,0.0304,71712662130
1,Vanguard 500 Index Fund,172326589,2022-12-30,0.0232,54537920779
2,Fidelity 500 Index Fund,81720092,2023-02-27,0.0110,25862775613
3,SPDR S&P 500 ETF Trust,80591530,2023-03-30,0.0108,25505608299
4,"Invesco ETF Tr-Invesco QQQ Tr, Series 1 ETF",75130340,2023-03-30,0.0101,23777250828
5,iShares Core S&P 500 ETF,66532494,2023-02-27,0.0089,21056204432
6,Vanguard Growth Index Fund,62920006,2022-12-30,0.0085,19912924190
7,Vanguard Institutional Index Fund-Institutiona...,52237514,2022-12-30,0.0070,16532129004
8,Growth Fund Of America Inc,41256784,2023-03-30,0.0055,13056947453
9,Vanguard Information Technology Index Fund,34018479,2022-11-29,0.0046,10766168607


In [28]:
# show news
msft.news

[{'uuid': '5ab07694-9da7-320d-bf6b-a51044b80e44',
  'title': '10 Stocks ChatGPT Said Will Make Me Rich in 10 Years',
  'publisher': 'Insider Monkey',
  'link': 'https://finance.yahoo.com/news/10-stocks-chatgpt-said-rich-202351994.html',
  'providerPublishTime': 1692563031,
  'type': 'STORY',
  'thumbnail': {'resolutions': [{'url': 'https://s.yimg.com/uu/api/res/1.2/RSsw9W8FGsUdwBxPOytsEg--~B/aD00NjQwO3c9Njk2MDthcHBpZD15dGFjaHlvbg--/https://media.zenfs.com/en/insidermonkey.com/25b9cf64bdd8c5617210ec66cab60576',
     'width': 6960,
     'height': 4640,
     'tag': 'original'},
    {'url': 'https://s.yimg.com/uu/api/res/1.2/86bZBdgsIF0A38CWHYQ46g--~B/Zmk9ZmlsbDtoPTE0MDtweW9mZj0wO3c9MTQwO2FwcGlkPXl0YWNoeW9u/https://media.zenfs.com/en/insidermonkey.com/25b9cf64bdd8c5617210ec66cab60576',
     'width': 140,
     'height': 140,
     'tag': '140x140'}]},
  'relatedTickers': ['MSFT', 'JNJ', 'AAPL', 'PG', 'V', 'NVDA']},
 {'uuid': 'a2989796-4d92-3b50-8d31-d7cf2f57be1f',
  'title': '10 Stocks That 

In [29]:
#multiple tickers

In [30]:
tickers = yf.Tickers('msft aapl goog')


In [31]:
tickers

yfinance.Tickers object <MSFT,AAPL,GOOG>

In [32]:
tickers.tickers['MSFT'].info


{'address1': 'One Microsoft Way',
 'city': 'Redmond',
 'state': 'WA',
 'zip': '98052-6399',
 'country': 'United States',
 'phone': '425 882 8080',
 'website': 'https://www.microsoft.com',
 'industry': 'Software—Infrastructure',
 'industryDisp': 'Software—Infrastructure',
 'sector': 'Technology',
 'sectorDisp': 'Technology',
 'longBusinessSummary': 'Microsoft Corporation develops and supports software, services, devices and solutions worldwide. The Productivity and Business Processes segment offers office, exchange, SharePoint, Microsoft Teams, office 365 Security and Compliance, Microsoft viva, and Microsoft 365 copilot; and office consumer services, such as Microsoft 365 consumer subscriptions, Office licensed on-premises, and other office services. This segment also provides LinkedIn; and dynamics business solutions, including Dynamics 365, a set of intelligent, cloud-based applications across ERP, CRM, power apps, and power automate; and on-premises ERP and CRM applications. The Int

In [33]:
tickers.tickers['AAPL'].history(period="1mo")


,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2023-07-19 00:00:00-04:00,192.839596,197.962667,192.390191,194.836899,80507300,0.00,0.0
2023-07-20 00:00:00-04:00,194.826901,196.205045,192.240397,192.869553,59581200,0.00,0.0
2023-07-21 00:00:00-04:00,193.838258,194.707080,190.972118,191.681168,71917800,0.00,0.0
2023-07-24 00:00:00-04:00,193.149180,194.647157,191.990741,192.490067,45377800,0.00,0.0
2023-07-25 00:00:00-04:00,193.069285,194.177788,192.659834,193.358887,37283200,0.00,0.0
2023-07-26 00:00:00-04:00,193.408819,195.376163,193.059300,194.237701,47471900,0.00,0.0
2023-07-27 00:00:00-04:00,195.755654,196.934055,192.290332,192.959427,47460200,0.00,0.0
2023-07-28 00:00:00-04:00,194.407479,196.364842,193.878195,195.565918,48291400,0.00,0.0
2023-07-31 00:00:00-04:00,195.795600,196.225028,194.996676,196.185074,38824100,0.00,0.0


In [34]:
tickers.tickers['GOOG'].actions


,Dividends,Stock Splits
Date,,
2014-03-27 00:00:00-04:00,0.0,2.002000
2015-04-27 00:00:00-04:00,0.0,1.002746
2022-07-18 00:00:00-04:00,0.0,20.000000


In [35]:
# to download the price data

In [36]:
data = yf.download("SPY AAPL", period="1mo")


[*********************100%%**********************]  2 of 2 completed


In [37]:
data

Adj Close                   Close                    High  \
                  AAPL         SPY        AAPL         SPY        AAPL   
Date                                                                     
2023-07-19  194.836899  455.200012  195.100006  455.200012  198.229996   
2023-07-20  192.869553  452.179993  193.130005  452.179993  196.470001   
2023-07-21  191.681168  452.179993  191.940002  452.179993  194.970001   
2023-07-24  192.490067  454.200012  192.750000  454.200012  194.910004   
2023-07-25  193.358887  455.440002  193.619995  455.440002  194.440002   
2023-07-26  194.237701  455.510010  194.500000  455.510010  195.639999   
2023-07-27  192.959427  452.489990  193.220001  452.489990  197.199997   
2023-07-28  195.565918  456.920013  195.830002  456.920013  196.630005   
2023-07-31  196.185074  457.790009  196.449997  457.790009  196.490005   
2023-08-01  195.346207  456.480011  195.610001  456.480011  196.729996   
2023-08-02  192.320297  450.130005  192.580002  450.130005  195.179993   
2023-08-03  190.912201  448.839996  191.169998  448.839996  192.369995   
2023-08-04  181.744583  446.809998  181.990005  446.809998  187.380005   
2023-08-07  178.608810  450.709991  178.850006  450.709991  183.130005   
2023-08-08  179.557526  448.750000  179.800003  448.750000  180.270004   
2023-08-09  177.949707  445.750000  178.190002  445.750000  180.929993   
2023-08-10  177.729996  445.910004  177.970001  445.910004  180.750000   
2023-08-11  177.789993  445.649994  177.789993  445.649994  178.619995   
2023-08-14  179.460007  448.109985  179.460007  448.109985  179.690002   
2023-08-15  177.449997  442.890015  177.449997  442.890015  179.479996   
2023-08-16  176.570007  439.640015  176.570007  439.640015  178.539993   
2023-08-17  174.000000  436.290009  174.000000  436.290009  177.509995   
2023-08-18  174.490005  436.500000  174.490005  436.500000  175.100006   

                               Low                    Open              \
                   SPY        AAPL         SPY        AAPL         SPY   
Date                                                                     
2023-07-19  456.429993  192.649994  454.109985  193.100006  455.010010   
2023-07-20  455.100006  192.500000  451.440002  195.089996  454.170013   
2023-07-21  454.170013  191.229996  452.170013  194.100006  453.959991   
2023-07-24  455.040009  192.250000  452.299988  193.410004  453.369995   
2023-07-25  456.739990  192.919998  453.869995  193.330002  453.920013   
2023-07-26  456.989990  193.320007  453.380005  193.669998  454.470001   
2023-07-27  459.440002  192.550003  451.549988  196.020004  459.019989   
2023-07-28  457.779999  194.139999  452.489990  194.669998  455.880005   
2023-07-31  458.160004  195.259995  456.049988  196.059998  457.410004   
2023-08-01  457.250000  195.279999  455.489990  196.240005  456.269989   
2023-08-02  453.519989  191.850006  449.350006  195.039993  453.250000   
2023-08-03  450.790009  190.690002  447.369995  191.570007  448.040009   
2023-08-04  452.899994  181.919998  446.269989  185.520004  450.720001   
2023-08-07  450.869995  177.350006  447.989990  182.130005  448.709991   
2023-08-08  450.700012  177.580002  445.269989  179.690002  448.079987   
2023-08-09  449.200012  177.009995  444.959991  180.869995  449.029999   
2023-08-10  451.700012  177.600006  444.700012  179.479996  448.190002   
2023-08-11  446.700012  176.550003  443.350006  177.320007  443.970001   
2023-08-14  448.109985  177.309998  444.380005  177.970001  444.700012   
2023-08-15  446.640015  177.050003  442.299988  178.880005  446.269989   
2023-08-16  444.179993  176.500000  439.529999  177.130005  442.459991   
2023-08-17  441.429993  173.479996  435.750000  177.139999  441.160004   
2023-08-18  437.570007  171.960007  433.010010  172.300003  433.369995   

               Volume             
                 AAPL        SPY  
Date                              
2023-07-19   80507300   65891700  
2023-07-20   59581200  